In [72]:
from lexnorm.definitions import DATA_PATH
import os
import pandas as pd

train = pd.read_csv(os.path.join(DATA_PATH, "hpc/train_annotated.txt"), index_col=0)
dev = pd.read_csv(os.path.join(DATA_PATH, "hpc/dev_annotated.txt"), index_col=0)

In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [70]:
train.fillna(0, inplace=True)
dev.fillna(0, inplace=True)
X_train = train.drop(["correct", "raw_tok_index", "gold"], axis=1)
X_dev = dev.drop(["correct", "raw_tok_index", "gold"], axis=1)
y_train = train["correct"]
y_dev = dev["correct"]

In [55]:
from sklearn import preprocessing

feature_vals = X_train.values
scaler = preprocessing.StandardScaler().fit(feature_vals)
feature_vals = scaler.transform(feature_vals)
X_train[:] = feature_vals

feature_vals = X_dev.values
scaler = preprocessing.StandardScaler().fit(feature_vals)
feature_vals = scaler.transform(feature_vals)
X_dev[:] = feature_vals

In [5]:
svm = LinearSVC(class_weight="balanced", verbose=1).fit(X_train, y_train.values.ravel())

[LibLinear]....................................................................................................
optimization finished, #iter = 1000

Using -s 2 may be faster (also see FAQ)

Objective value = -77126.360842
nSV = 259470


/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [11]:
from joblib import dump, load
dump(svm, os.path.join(DATA_PATH, "../models/svm.joblib"))
test = load(os.path.join(DATA_PATH, "../models/svm.joblib"))
test.coef_

array([[ 0.17172243, -0.13962874, -0.20333317,  0.12403685,  0.00285971,
         1.8674295 , -0.0935477 ,  0.16077809, -0.05016316,  0.14621954,
        -0.64900722, -0.14582517,  0.        ,  0.08600042]])

In [64]:
preds = svm.decision_function(X_dev)

In [65]:
svm.score(X_dev, y_dev)

0.9983700596728575

In [66]:
preds

array([-1.43408099, -1.56879485, -1.4699958 , ..., -2.39990597,
       -2.10414587, -3.19460247])

In [73]:
dev["preds"] = preds
dev

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length,correct,raw_tok_index,gold,preds
an,0.377180,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2,NaN,72.0,1.0,1.0,3.0,NaN,35_0,can,-1.434081
ca,0.255824,NaN,NaN,NaN,NaN,NaN,4.0,1.0,2,NaN,72.0,1.0,1.0,3.0,NaN,35_0,can,-1.568795
cab,0.261057,NaN,NaN,NaN,NaN,NaN,1.0,1.0,3,NaN,72.0,1.0,1.0,3.0,NaN,35_0,can,-1.469996
cam,0.299136,NaN,NaN,NaN,NaN,NaN,2.0,1.0,3,NaN,72.0,1.0,1.0,3.0,NaN,35_0,can,-1.476411
can,1.000000,NaN,1.0,1.0,NaN,72.0,NaN,1.0,3,1.0,72.0,1.0,1.0,3.0,1.0,35_0,can,9.580333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lulu,0.401415,NaN,NaN,NaN,NaN,NaN,14.0,1.0,4,NaN,205.0,1.0,1.0,3.0,NaN,32_160,lol,-2.435480
pol,0.230619,NaN,NaN,NaN,NaN,NaN,9.0,1.0,3,NaN,205.0,1.0,1.0,3.0,NaN,32_160,lol,-2.371043
smh,0.896277,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,205.0,1.0,1.0,3.0,NaN,32_160,lol,-2.399906
sol,0.285978,NaN,NaN,NaN,NaN,NaN,3.0,1.0,3,NaN,205.0,1.0,1.0,3.0,NaN,32_160,lol,-2.104146


In [74]:
# get top prediction for each raw token
dev.sort_values("preds", ascending=False).drop_duplicates(["raw_tok_index"])

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length,correct,raw_tok_index,gold,preds
the,1.0,NaN,1.0,1.0,NaN,515.0,NaN,1.0,3,1.0,515.0,1.0,1.0,3.0,1.0,55_92,the,64.457622
the,1.0,NaN,1.0,1.0,NaN,515.0,NaN,1.0,3,1.0,515.0,1.0,1.0,3.0,1.0,32_108,the,64.457622
the,1.0,NaN,1.0,1.0,NaN,515.0,NaN,1.0,3,1.0,515.0,1.0,1.0,3.0,1.0,49_79,the,64.457622
the,1.0,NaN,1.0,1.0,NaN,515.0,NaN,1.0,3,1.0,515.0,1.0,1.0,3.0,1.0,39_83,the,64.457622
the,1.0,NaN,1.0,1.0,NaN,515.0,NaN,1.0,3,1.0,515.0,1.0,1.0,3.0,1.0,25_110,the,64.457622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pun,1.0,NaN,1.0,1.0,NaN,NaN,16.0,1.0,3,1.0,NaN,1.0,1.0,3.0,1.0,23_115,pun,0.068293
dun,1.0,NaN,1.0,1.0,NaN,NaN,16.0,1.0,3,1.0,NaN,1.0,1.0,3.0,NaN,2_24,don't,0.068293
nap,1.0,NaN,1.0,1.0,NaN,NaN,16.0,1.0,3,1.0,NaN,1.0,1.0,3.0,1.0,23_75,nap,0.068293
fog,1.0,NaN,1.0,1.0,NaN,NaN,16.0,1.0,3,1.0,NaN,1.0,1.0,3.0,1.0,29_56,fog,0.068293
